# Module 3: Regression and correlation


## Nonlinear regression

The relative diffusivity (compared to water) of various sized solutes through a 5% agarose hydrogel was found experimentally. The experimental data is shown in the table below. As expected, the diffusivity through the porous hydrogel decreases as the size of the solute increases (radius = a). Our goal is to estimate the pore size of the hydrogel using the experiemental data.

In [2]:
import scipy.stats as stats
from scipy.optimize import curve_fit  # New stats package!
import numpy as np
import plotly.graph_objects as go
import pandas as pd

df = pd.read_csv("https://raw.githubusercontent.com/jlongc12/stats_py/main/data/solute_data.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   solute  5 non-null      object 
 1   radius  5 non-null      float64
 2   Dratio  5 non-null      float64
dtypes: float64(2), object(1)
memory usage: 248.0+ bytes


## Curve fitting
The Renkin equation, shown below, describes diffusivity in terms of the pore size (r) and solute size (a). Use the curve fitting package from scipy to fit the data to the Renkin equation. What is the estimated pore size?

$\frac{D_{Gel}}{D_{H2O}} = \Big(1-\frac{a}{r}\Big)^2\left(1-2.1\Big(\frac{a}{r}\Big)+2.09\Big(\frac{a}{r}\Big)^3-0.95\Big(\frac{a}{r}\Big)^5\right)$



In [3]:
# Write a function that calculates diffusivity as a function of pore size and solute size
def func(a,r):
    return np.power(1-a/r,2)*(1-2.1*a/r+2.09*np.power(a/r,3)-0.95*np.power(a/r,5))

# Use the new curve_fit function. What are the outputs? Check the documentation.
r, rcov = curve_fit(func,df['radius'],df['Dratio'])
r_nm = r*1e9

print('Pore size: %.2f nm' % r_nm)

Pore size: 659.57 nm


Plot your fitted equation with the solute data provided. Is the Renkin equation a good fit?

In [ ]:
# Again, plotting can get complicated, so we've included the code for you. Note
# how both plot types use the go.Scatter call, but with different 'mode' types.

fig = go.Figure()
fig.add_trace(go.Scatter(x=df['radius'],
                         y=df['Dratio'],
                         mode="markers",
                         name="data"))    # Label your trace for the legend

linex = np.linspace(3e-8,4e-7,100)        # use np.linspace to define a vector of x-values
fig.add_trace(go.Scatter(x=linex,
                         y=func(linex,r),
                         mode="lines",
                         name="fit"))
fig.show()

## Residuals

Calculate and plot the residuals for each data point. What does this plot tell us about the fit? Verify your answer by calculating the R2 value.

In [ ]:
Dratio_fit = func(df['radius'],r)
residuals = df['Dratio']-Dratio_fit

fig = go.Figure()
fig.add_trace(go.Scatter(x=df['radius'],
                         y=residuals,
                         mode="markers"))
fig.show()

# There's a couple steps here. Check your work!
sse = np.sum(np.square(residuals))
solute_mean = np.mean(df['Dratio'])
sst = np.sum(np.square(df['Dratio']-df['Dratio'].mean()))
R2 = 1-sse/sst
print('R2: %.2f' % R2)

R2: 0.98


Calculate the 95% confidence interval for your fitted variable, r. Plot the 95% prediction bounds of your fit. What do these bounds indicate?



In [ ]:
# It's not immediately obvious how to get the standard deviation, so we've included it here.
# Use this to find your upper and lower bounds. Hint: you will need stats.t.ppf
s = np.sqrt(np.diag(rcov))
dof = len(df['Dratio'])-1
alpha = 0.05
tval = stats.t.ppf(1-alpha/2, dof) 
rlow = r-s*tval
rhigh = r+s*tval

fig = go.Figure()
fig.add_trace(go.Scatter(x=df['radius'],
                         y=df['Dratio'],
                         mode="markers",
                         name="data"))

linex = np.linspace(3e-8,4e-7,100)
fig.add_trace(go.Scatter(x=linex,
                         y=func(linex,r),
                         mode="lines",
                         name="fit"))

# Repeat the block above for your low and high bounds for r
linex = np.linspace(3e-8,4e-7,100)
fig.add_trace(go.Scatter(x=linex,
                         y=func(linex,rlow),
                         mode="lines",
                         name="lower prediction"))

linex = np.linspace(3e-8,4e-7,100)
fig.add_trace(go.Scatter(x=linex,
                         y=func(linex,rhigh),
                         mode="lines",
                         name="upper prediction"))
fig.show()